In [1]:
import os
import pandas as pd
import spacy
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English # Import the English language class

nlp = spacy.load('en_core_web_sm')
#nlp = English()
#nlp = spacy.laod('en')

path = 'data/'
arr = os.listdir(path)
for filename in arr:
    if filename.endswith('.txt'):
        xpath = os.path.join(path,filename)
        with open(xpath, 'r') as file:
            contents = file.read()
            doc = nlp(contents)
            entities = [(ent.label_, ent.text, filename) for ent in doc.ents]
            df = pd.DataFrame(entities, columns =['type', 'text', 'filename']) 

In [2]:
import pandas as pd
import os
import nltk
from nltk.tokenize import PunktSentenceTokenizer
from nltk.corpus import reuters

tokenizer = PunktSentenceTokenizer()
path = 'data/'
arr = os.listdir(path)
data = []
for filename in arr:
    if filename.endswith('.txt'):
        xpath = os.path.join(path,filename)
        with open(xpath, 'r') as file:
            contents = file.read()
            data.extend(tokenizer.tokenize(contents))
            

In [3]:
df = pd.DataFrame(data, columns =['text']) 
df.to_excel('data/punktsentences.xlsx')
df.to_csv('data/punktsenteces.csv',index=False)

In [4]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline

In [6]:
data = pd.read_csv('data/markedsentences.csv', )
data.head()

,relevant,type,sentence
0,0,0,monson.
1,0,0,"maine history 1822 - 1972 history of monson, m..."
2,0,0,this history could not have been compiled with...
3,0,0,the main sources of information for this book ...
4,0,0,may this history bring pleasure to the readers...


In [7]:
data.shape

(1350, 3)

In [8]:
data.relevant.value_counts()

0    1320
1      30
Name: relevant, dtype: int64

In [9]:
data["tokenized"] = data.apply(lambda row: nltk.word_tokenize(row["sentence"]), axis=1)
data.to_csv('data/marked.csv', index = False)

In [10]:
import spacy
import string

from spacy.lang.en.stop_words import STOP_WORDS
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English # Import the English language class
from spacy.matcher import Matcher

In [11]:
punctuations = string.punctuation# Create our list of punctuation marks
nlp = spacy.load('en') # Create list of stopwords
stop_words = spacy.lang.en.stop_words.STOP_WORDS
parser = English()# Load English tokenizer, tagger, parser, NER and word vectors

def spacy_tokenizer(sentence):# Creating tokenizer function
    
    mytokens = parser(sentence)# Creating token object, used to create documents with linguistic annotations.
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ] # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ] # Removing stop words
    
    return mytokens# return preprocessed list of tokens

# Custom transformer using spaCy
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        # Cleaning Text
        return [clean_text(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}

# Basic function to clean the text
def clean_text(text):
    # Removing spaces and converting text into lowercase
    return text.strip().lower()



In [12]:
bow_vector = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1))

In [14]:
tfidf_vector = TfidfVectorizer(tokenizer = spacy_tokenizer)

In [15]:
from sklearn.model_selection import train_test_split

X = data['sentence'] # the features we want to analyze
ylabels = data['relevant'] # the labels, or answers, we want to test against

X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.3)

### Classifiers - Regression, TFIDF,  Word2Vec mean

In [16]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()

# Create pipeline using Bag of Words
pipe = Pipeline([('cleaner', predictors()),
                 ('vectorizer', bow_vector),
                 ('classifier', classifier)])

# Create Average Vector score


# model generation
pipe.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('cleaner', <__main__.predictors object at 0x0000012023E4D860>),
                ('vectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function spacy_tokenizer at 0x00000120208C0A60>,
                                 vocabulary=None)),
                ('classifier',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_

In [17]:
from sklearn import metrics
# Predicting with a test dataset
predicted = pipe.predict(X_test)

# Model Accuracy
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted))

Logistic Regression Accuracy: 0.9703703703703703
Logistic Regression Precision: 0.0
Logistic Regression Recall: 0.0


C:\Users\Lucky\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### Vectorize and Train

In [59]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline

In [63]:
training = pd.read_csv('data/marked.csv')
training

,relevant,type,sentence,tokenized
0,0,0,monson.,"['monson', '.']"
1,0,0,"maine history 1822 - 1972 history of monson, m...","['maine', 'history', '1822', '-', '1972', 'his..."
2,0,0,this history could not have been compiled with...,"['this', 'history', 'could', 'not', 'have', 'b..."
3,0,0,the main sources of information for this book ...,"['the', 'main', 'sources', 'of', 'information'..."
4,0,0,may this history bring pleasure to the readers...,"['may', 'this', 'history', 'bring', 'pleasure'..."
...,...,...,...,...
1345,0,0,roebuck and co.,"['roebuck', 'and', 'co', '.']"
1346,0,0,gene johnson 42 main street authorized catalog...,"['gene', 'johnson', '42', 'main', 'street', 'a..."
1347,0,0,"15, monson, maine telephone 997-3327 history o...","['15', ',', 'monson', ',', 'maine', 'telephone..."
1348,0,0,chase 8: klmball oil company large enough to p...,"['chase', '8', ':', 'klmball', 'oil', 'company..."


#### -------- Text Classification With Word2Vec --------

In [76]:
#Vectorizing text
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

#GenSim
from gensim.models import Word2Vec
import numpy as np
import gensim
from gensim import corpora
from pprint import pprint

#instantiate CountVectorizer()
cv=CountVectorizer()

training.score = [] 

# count matrix
data["count"] = data.apply(lambda row: cv.transform(row["tokenized"]), axis=1)
 
# tf-idf scores
data["tfidf"] = data.apply(lambda row: tfidf_transformer.transform(row["count"]), axis=1)

#get tfidf vector
data["tfidfvector"] = data.apply(lambda row: tf_idf_vector([0]), axis=1) 


NotFittedError: Vocabulary not fitted or provided